
PM Report Filter
---
The goal of this document is to assist building a stand alone python file that can filter an excel report [*my current employer*] provides.
    
Currently I manually filter this report twice a week and transfer the information to a Google Sheet where the information is used to flag equipment currently in use are due for PM (Preventive Maint.)
    
### Goals 
---

- Set up functions to streamline cleaning process - Completed
-Output an excel file with a PM List for trailers over 93% Due - Completed
-Output an excel file with a PM List just for PC117 Trailers - Completed
-Output both files into one excel file with multiple sheets - Completed
-Create a standalone executable python file

### Stretch Goal:
---
Implement a way to output directly to Google Sheets

In [1]:
import pandas as pd

In [2]:
dawg = pd.read_excel("./PM.xlsx")

In [3]:
#Functions
def sort_trls(df):
    """Takes the PM List and sorts trailers and Due PM Percentages
       Output: Rollerbed, PM Highest to Lowest
               Trailer, PM Highest to Lowest"""
    rb = df[df["Unit Type"] == "ROLLERBED"].sort_values(by="Due Percent", ascending=False)
    trl = df[df["Unit Type"] == "TRAILER"].sort_values(by="Due Percent", ascending=False)
    df = pd.concat([rb, trl], ignore_index = True)
    return df

def clean_dawg(df):
    """Cleans the DAWG Report.
       -Fills NA Values, Sets CC to Int, Removes PM's < 94%, Removes CUST Trailers"""
    df["Last Touched Cost Center"] = df["Last Touched Cost Center"].fillna(0)
    df["Last Touched Cost Center"] = df["Last Touched Cost Center"].astype('int64')
    df = df[df["Due Percent"]>93]
    df = df[df["Unit Type"] != "CUST TRLR"]
    df = sort_trls(df)
    return df

def pc117_pm(df):
    """Filters the DAWG Report to only return the trailers for PC117
       -RB's for PC117 & PC854, All PC117 Trailers, and PC117 Last Touch Trailers"""
    rb = df[(df["Cost Center"].isin([117,854])) & (df["Unit Type"] == "ROLLERBED")]
    cc = df[(df["Cost Center"]==117) & (df["Unit Type"] != "ROLLERBED")]
    lt = df[(df["Last Touched Cost Center"]==117) & (df["Cost Center"]!=117)]
    df = pd.concat([rb, cc, lt], ignore_index = True)
    return df

In [4]:
#Clean the DAWG Report, set up the two sheets
dawg = clean_dawg(dawg)
pms = pc117_pm(dawg)

#Write to the Excel Sheets
writer = pd.ExcelWriter("./PMTest.xlsx", engine="xlsxwriter")
pms.to_excel(writer, sheet_name = "PC117", index=False)
dawg.to_excel(writer, sheet_name = "PMList", index=False)
writer.save()